In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp

In [84]:
df = pd.read_csv(filepath_or_buffer='./analysis_df.csv', delimiter=',')
emp_df = pd.read_csv(filepath_or_buffer='./analysis_df_employee.csv', delimiter=',')

In [85]:
df.head(5)

,Age,Gender,Salary,Title,Formal Education,Learning Platforms,Coding Experience,ML Experience,Important Activities,Languages,...,Published Papers,ML Research,ML Serve,Ethical AI tools,Industry Type,Data Storage Products,Data Products,ML Monitor Tools,Country,Year
0,18-21,Man,1528.0,Student,Bachelor's degree,Coursera -- -1,< 1 year,< 1 year,Build and/or run a machine learning service th...,Python -- Java -- C/C++ -- -1,...,NaN,NaN,NaN,NaN,I am a student,NaN,NaN,NaN,Viet Nam,2018
1,18-21,Man,1528.0,Student,Some college/university study without<br>earni...,Udacity -- Coursera -- edX -- Udemy -- -1,< 1 year,< 1 year,Analyze and understand data to influence produ...,Python -- -1,...,NaN,NaN,NaN,NaN,Computers/Technology,NaN,NaN,NaN,Viet Nam,2018
2,18-21,Man,NaN,Student,Bachelor's degree,Coursera -- -1,1-5 years,1-2 years,Build and/or run a machine learning service th...,Python -- -1,...,NaN,NaN,NaN,NaN,Computers/Technology,NaN,NaN,NaN,Viet Nam,2018
3,22-24,Man,1528.0,Student,Master's degree,Coursera -- DataQuest -- -1,< 1 year,< 1 year,Do research that advances the state of the art...,Python -- R -- Bash -- -1,...,NaN,NaN,NaN,NaN,I am a student,NaN,NaN,NaN,Viet Nam,2018
4,22-24,Man,1528.0,Data Scientist,Bachelor's degree,Coursera -- -1,1-5 years,< 1 year,Analyze and understand data to influence produ...,R -- SQL -- -1,...,NaN,NaN,NaN,NaN,Insurance/Risk Assessment,NaN,NaN,NaN,Viet Nam,2018


In [122]:
df['Coding Experience'].value_counts().sort_values()

20+ years                     2130
I have never written code     2776
10-20 years                   3027
5-10 years                    4999
< 1 year                     11422
1-5 years                    22251
Name: Coding Experience, dtype: int64

In [141]:
coding_exp_counts = df['Coding Experience'].value_counts()

avg_salary = df.groupby('Coding Experience')['Salary'].mean().sort_values()

fig = go.Figure(data=[
    go.Pie(labels=coding_exp_counts.index, values=coding_exp_counts.values),
])

fig.update_layout(
    title='Coding Experience and Salary',
    grid=dict(rows=1, columns=2),
    template='plotly_white'
)

fig.add_trace(go.Bar(x=avg_salary.index, y=avg_salary.values))

fig.show()